In [5]:
!pip install -q boto3 crewai crewai_tools langchain-aws

In [6]:
from crewai import Agent, Task, Crew

In [16]:
import boto3
from langchain_aws import ChatBedrock

boto_session = boto3.Session(profile_name="sandbox", region_name="us-west-2")
bedrock_runtime=boto_session.client(service_name="bedrock-runtime")

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

model_kwargs =  { 
    "max_tokens": 512,
    "temperature": 0.1,
    "top_k": 250,
    "top_p": 1,
}

llm = ChatBedrock(
    client=bedrock_runtime,
    model_id=model_id,
    model_kwargs=model_kwargs,
)


In [7]:
import os
import getpass
os.environ["SERPER_API_KEY"] = getpass.getpass("Serper API key:")

Serper API key: ········


In [8]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [17]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [19]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [20]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [21]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [22]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [23]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [24]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

In [25]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [26]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

In [27]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AMZN',
    'initial_capital': '10000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [28]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AMZN). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
Thought: To effectively monitor and analyze the market data for AMZN stock, I will need to delegate tasks to my team members and gather relevant information from them.

Action: Delegate work to co-worker
Action Input: {
    "task": "Collect and preprocess historical market data for AMZN stock, including stock prices, trading volumes, and relevant financial news or events.",
    "context": "We need to continuously monitor and analyze the market data for AMZN stock. To do this effectively, we require a comprehensive dataset of historical market data, including stock prices, trading volumes, and any relevant financial news or events that may have impacted the stock's performance. This data should be preprocessed and 

In [29]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Trading AMZN Stock

I. Breakout Trading Strategy Risks and Mitigation Strategies

A. Market Risk:
   - AMZN is a large-cap stock, and its price movements can be influenced by various market factors, such as overall market sentiment, economic conditions, and industry trends.
   - Sudden market shifts or unexpected events can lead to significant price fluctuations, potentially triggering false breakout signals or causing losses if positions are not managed properly.
   - Mitigation: Continuously monitor market conditions, news, and events that may impact AMZN's price movements, and be prepared to adjust your trading strategy or exit positions if necessary.

B. Volatility Risk:
   - AMZN is known for its high volatility, which means that its price can experience substantial swings within a short period.
   - High volatility can lead to wider stop-loss levels, increasing the potential for larger losses if the trade goes against you.
   - Whipsaw price movements, where the stock breaks out but quickly reverses, can lead to false entry signals.
   - Mitigation: Implement proper position sizing and risk management strategies, such as using stop-loss orders and limiting the risk per trade to a predetermined percentage of your trading capital.

C. Execution Risk:
   - Due to AMZN's high liquidity and trading volume, slippage may not be a significant concern under normal market conditions.
   - However, during periods of high volatility or news events, there could be temporary liquidity gaps, making it challenging to execute orders at desired prices.
   - Mitigation: Utilize limit orders and avoid market orders during periods of high volatility or liquidity constraints.

D. Psychological Risk:
   - Breakout trading strategies can be emotionally challenging, as traders may experience fear of missing out (FOMO) or the temptation to hold positions for too long, leading to potential losses.
   - Mitigation: Maintain discipline and adhere to